In [1]:
import pandas as pd
df = pd.read_csv("AG.txt", sep="\t", low_memory=False)
print(df.head())
print(df.shape)
print(df.columns[:20])

           #SampleID BarcodeSequence LinkerPrimerSequence CARBOHYDRATE_PER  \
0  000011224.1210932         no_data              no_data               55   
1  000005034.1076350         no_data              no_data               56   
2  000011351.1210669         no_data              no_data               16   
3  000002013.1131550         no_data              no_data          no_data   
4  000004731.1076409         no_data              no_data               61   

  NONFOODALLERGIES_DRUG NONFOODALLERGIES_BEESTINGS ASSIGNED_FROM_GEO RUN_DATE  \
0               no_data                    no_data                 n  no_data   
1               no_data                    no_data                 n  no_data   
2               no_data                    no_data                 n  no_data   
3               no_data                    no_data                 n  no_data   
4                   yes                    no_data                 n  no_data   

  PET_LOCATIONS   AGE  ... LIBRARY_CONSTRUCT

In [2]:
pip install biom-format

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 8.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.0 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from biom import load_table
table = load_table('AG_100nt_even10k.biom')
print(table.shape)
print(table.ids())  # sample IDs

(18089, 3107)
['000007117.1075649' '000005636.1053788' '000005637.1053909' ...
 '000011980.1210764' '000005567.1131812' '000003557.1130030']


In [5]:
import pandas as pd
from biom import load_table

# Load biom table
table = load_table('AG_100nt_even10k.biom')

# Convert BIOM to pandas DataFrame
otu_df = table.to_dataframe(dense=True).T  # transpose so rows = samples

# Load metadata
url = "https://raw.githubusercontent.com/biocore/American-Gut/refs/heads/master/data/AG/AG.txt"
meta_df = pd.read_csv(url, sep='\t')

# Clean metadata
meta_df = meta_df.replace('no_data', pd.NA)

# Merge (only keep samples present in both)
merged = meta_df.merge(otu_df, left_on="#SampleID", right_index=True, how="inner")

print(merged.shape)
print(merged.columns[:10])


/var/folders/b5/yh6_b6cs4vgfqd16nkxmhl9h0000gn/T/ipykernel_75975/1948684792.py:12: DtypeWarning: Columns (34,197) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(url, sep='\t')


(3107, 18291)
Index(['#SampleID', 'BarcodeSequence', 'LinkerPrimerSequence',
       'CARBOHYDRATE_PER', 'NONFOODALLERGIES_DRUG',
       'NONFOODALLERGIES_BEESTINGS', 'ASSIGNED_FROM_GEO', 'RUN_DATE',
       'PET_LOCATIONS', 'AGE'],
      dtype='object')


In [6]:
meta_ids = set(meta_df['#SampleID'])
biom_ids = set(table.ids())
print(len(meta_ids & biom_ids), "samples in common")


3107 samples in common


In [8]:
from biom import load_table

# Load the BIOM table
table = load_table("AG_100nt_even10k.biom")

# Extract OTU IDs
otu_ids = table.ids(axis='observation')

# Extract taxonomy info
otu_metadata = table.metadata(axis='observation')  # this contains taxonomic info

# Example: print first 5 OTUs with their taxonomy
for i in range(5):
    print(otu_ids[i], otu_metadata[i])


988534 defaultdict(<function Table._cast_metadata.<locals>.cast_metadata.<locals>.<lambda> at 0x10e9acfe0>, {'taxonomy': ['k__Bacteria', 'p__Proteobacteria', 'c__Betaproteobacteria', 'o__Neisseriales', 'f__Neisseriaceae', 'g__', 's__']})
4479944 defaultdict(<function Table._cast_metadata.<locals>.cast_metadata.<locals>.<lambda> at 0x10e9ad120>, {'taxonomy': ['k__Bacteria', 'p__Actinobacteria', 'c__MB-A2-108', 'o__0319-7L14', 'f__', 'g__', 's__']})
370251 defaultdict(<function Table._cast_metadata.<locals>.cast_metadata.<locals>.<lambda> at 0x10e9ad1c0>, {'taxonomy': ['k__Bacteria', 'p__Proteobacteria', 'c__Gammaproteobacteria', 'o__Oceanospirillales', 'f__Endozoicimonaceae', 'g__Endozoicomonas', 's__montiporae']})
1126569 defaultdict(<function Table._cast_metadata.<locals>.cast_metadata.<locals>.<lambda> at 0x10e9ad260>, {'taxonomy': ['k__Bacteria', 'p__Proteobacteria', 'c__Deltaproteobacteria', 'o__Myxococcales', 'f__', 'g__', 's__']})
581609 defaultdict(<function Table._cast_metadata